# Mapping

- [Mappings and field types](https://opensearch.org/docs/latest/field-types/index/)
- [Supported field types](https://opensearch.org/docs/latest/field-types/supported-field-types/index/)

In [12]:
from datetime import datetime, timezone

from opensearchpy import OpenSearch
from rich import print as rprint

In [5]:
def create_oss_for_local_container(test_conn: bool = True) -> OpenSearch:
    """
    If you are using the OpenSearch Docker image for local developing following
    this https://hub.docker.com/r/opensearchproject/opensearch,
    and you are using this command to start the container:
    ``docker run -p 9200:9200 -p 9600:9600 -e "discovery.type=single-node" --name opensearch-node -d opensearchproject/opensearch:latest``
    then this function should give you the right OSS object.
    """
    oss = OpenSearch(
        hosts=[{"host": "localhost", "port": 9200}],
        http_auth=("admin", "admin"),
        http_compress=True,
        use_ssl=True,
        verify_certs=False,
        ssl_assert_hostname=False,
        ssl_show_warn=False,
        timeout=300,
    )
    if test_conn:
        res = oss.cat.indices(format="json")
        print(res)
    return oss

In [6]:
oss = create_oss_for_local_container()

[{'health': 'green', 'status': 'open', 'index': '.plugins-ml-config', 'uuid': 'ZrWp7o2hTm6kKF1mRJeuVg', 'pri': '1', 'rep': '0', 'docs.count': '1', 'docs.deleted': '0', 'store.size': '3.8kb', 'pri.store.size': '3.8kb'}, {'health': 'green', 'status': 'open', 'index': '.opensearch-observability', 'uuid': 'NWIZeV4pQue8t0eWMrDK1g', 'pri': '1', 'rep': '0', 'docs.count': '0', 'docs.deleted': '0', 'store.size': '208b', 'pri.store.size': '208b'}, {'health': 'green', 'status': 'open', 'index': '.opendistro_security', 'uuid': 'T2hapD0-SGWmhiBFuMNEAQ', 'pri': '1', 'rep': '0', 'docs.count': '10', 'docs.deleted': '0', 'store.size': '75.4kb', 'pri.store.size': '75.4kb'}]


## asf

In [9]:
index = "forum"

In [10]:
oss.indices.delete(index=index, ignore=[400, 404])

{'acknowledged': True}

In [11]:
res = oss.indices.create(
    index=index,
    body={
        "mappings": {
            "properties": {
                "title": {"type": "text"},
                "body": {"type": "text"},
                "create_at": {"type": "date"},
                "last_reply_at": {"type": "date"},
                "liked": {"type": "integer"},
            },
        },
    },
    ignore=400,
)
rprint(res)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'forum'}

In [16]:
res = oss.index(
    index=index,
    id="id-1",
    body={
        "title": "is OpenSearch fast on search?",
        "body": "yes it is really fast, even on Parabytes of documents.",
        "create_at": datetime(2023, 7, 15, tzinfo=timezone.utc),
        "last_reply_at": datetime(2023, 7, 21, tzinfo=timezone.utc),
        "liked": 20,
    },
)
rprint(res)

{
    '_index': 'forum',
    '_id': 'id-1',
    '_version': 1,
    'result': 'created',
    '_shards': {'total': 2, 'successful': 1, 'failed': 0},
    '_seq_no': 0,
    '_primary_term': 1
}

## Search

- [Search](https://opensearch.org/docs/latest/search-plugins/index/)
- [Query DSL](https://opensearch.org/docs/latest/query-dsl/index/)

In [20]:
query = {
    "query": {
        "match": {"title": "opensearch"}
    }
}
res = oss.search(
    index=index,
    body=query,
)
rprint(res)

{
    'took': 5,
    'timed_out': False,
    '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
    'hits': {
        'total': {'value': 1, 'relation': 'eq'},
        'max_score': 0.2876821,
        'hits': [
            {
                '_index': 'forum',
                '_id': 'id-1',
                '_score': 0.2876821,
                '_source': {
                    'title': 'is OpenSearch fast on search?',
                    'body': 'yes it is really fast, even on Parabytes of documents.',
                    'create_at': '2023-07-15T00:00:00+00:00',
                    'last_reply_at': '2023-07-21T00:00:00+00:00',
                    'liked': 20
                }
            }
        ]
    }
}

In [18]:
query = {
    "query": {
        "match": {"title": "unknown"}
    }
}
res = oss.search(
    index=index,
    body=query,
)
rprint(res)

{
    'took': 1,
    'timed_out': False,
    '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
    'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}
}

In [19]:
query = {
    "query": {
        "match": {"title": "opansearch"}
    }
}
res = oss.search(
    index=index,
    body=query,
)
rprint(res)

{
    'took': 1,
    'timed_out': False,
    '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
    'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}
}

In [22]:
query = {
    "query": {
        "match": {
            "title": {
                "query": "opansearch",
                "fuzziness": "AUTO",
            }
        }
    }
}
res = oss.search(
    index=index,
    body=query,
)
rprint(res)

{
    'took': 16,
    'timed_out': False,
    '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
    'hits': {
        'total': {'value': 1, 'relation': 'eq'},
        'max_score': 0.25891387,
        'hits': [
            {
                '_index': 'forum',
                '_id': 'id-1',
                '_score': 0.25891387,
                '_source': {
                    'title': 'is OpenSearch fast on search?',
                    'body': 'yes it is really fast, even on Parabytes of documents.',
                    'create_at': '2023-07-15T00:00:00+00:00',
                    'last_reply_at': '2023-07-21T00:00:00+00:00',
                    'liked': 20
                }
            }
        ]
    }
}

In [24]:
query = {
    "query": {
        "term": {
            "title": "is OpenSearch fast on search?"
        }
    }
}
res = oss.search(
    index=index,
    body=query,
)
rprint(res)

{
    'took': 3,
    'timed_out': False,
    '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
    'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}
}